In [1]:
%reload_ext autoreload

# Imports

import os
import sys
module_path = os.path.abspath(os.path.join('../image_gen_dm')) # or the path to your source code
sys.path.append(str(module_path))

import tensorflow as tf
import tf_keras as tfk
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

tfkl = tfk.layers
tfpl = tfp.layers
tfd = tfp.distributions

TF_ENABLE_ONEDNN_OPTS=0
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(TF_ENABLE_ONEDNN_OPTS)
os.environ['TG_GPU_ALLOCATOR'] = 'cuda_malloc_async'

import image_gen_vae as igvae
import image_gen_vae.constants as consts

print('Tensorflow Version:', tf.__version__)
print("Available GPUs: ", len(tf.config.list_physical_devices('GPU')))

2024-04-29 17:00:28.200366: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow Version: 2.16.1
Available GPUs:  1


2024-04-29 17:00:33.214251: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 17:00:33.243624: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 17:00:33.243683: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
# Constants

IMAGE_SIZE = 64

ENCODER_HIDDEN_SIZES = [
    [1792, 448],
    [1792, 448],
    [1792, 448],
    [1792, 448],

    [1792, 448],
    [1792, 448],
    [1792, 448],
    [1792, 448],

    [1792, 448],
    [1792, 448],
    [1792, 448],
    [1792, 448],

    [1792, 448],
    [1792, 448],
    [1792, 448],
    [1792, 448],
]

DECODER_HIDDEN_SIZES = [] # Construct from encoder hidden sizes in reverse

for hidden_sizes in ENCODER_HIDDEN_SIZES:
    DECODER_HIDDEN_SIZES.append(hidden_sizes[::-1])


ENCODER_CONV_CONFIGS = [
    [(112, 3, 2), (224, 3, 2), (448, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (224, 3, 2), (448, 5, 4), (1152, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (224, 3, 2), (576, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (224, 3, 2), (576, 5, 4), (1152, 3, 2), (2048, 3, 2)],

    [(112, 3, 2), (288, 3, 2), (448, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (288, 3, 2), (448, 5, 4), (1152, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (288, 3, 2), (576, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (288, 3, 2), (576, 5, 4), (1152, 3, 2), (2048, 3, 2)],

    [(144, 3, 2), (224, 3, 2), (448, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (224, 3, 2), (448, 5, 4), (1152, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (224, 3, 2), (576, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (224, 3, 2), (576, 5, 4), (1152, 3, 2), (2048, 3, 2)],

    [(144, 3, 2), (288, 3, 2), (448, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (288, 3, 2), (448, 5, 4), (1152, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (288, 3, 2), (576, 5, 4), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (288, 3, 2), (576, 5, 4), (1152, 3, 2), (2048, 3, 2)],
]

DECODER_CONV_CONFIGS = [] # Construct from encoder conv configs in reverse

for conv_configs in ENCODER_CONV_CONFIGS:
    DECODER_CONV_CONFIGS.append(conv_configs[::-1])

FLATTENED_SHAPES = [
    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),

    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),

    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),

    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),
    (1, 1, 2048),
]

FLATTENED_SIZES = [] # Construct from flattened shapes

for shape in FLATTENED_SHAPES:
    FLATTENED_SIZES.append(shape[0] * shape[1] * shape[2])

LATENT_DIM = 2

MODEL_NAMES = []

for i in range(len(ENCODER_HIDDEN_SIZES)):
    MODEL_NAMES.append(f'cvae{i+1+28}')

LEARNING_RATE = 1e-4

VAL_PERCENTAGE = 0.1

# Models to run

MODEL_START = 9 # Rasmus 0-7, Viktor 8-15
MODEL_COUNT = 1

In [3]:
# Dataset loading

train_ds, val_ds = igvae.utils.load_datasets(val_percentage=VAL_PERCENTAGE)

Loading datasets...


2024-04-29 17:00:33.375714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 17:00:33.375829: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 17:00:33.375890: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 17:00:33.551065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 17:00:33.551136: I external/local_xla/xla/stream_executor

Training Images:  8100
Evaluation Images:  900
Training Images (post-duplication):  8100
Validation Images (post-duplication):  900


In [4]:
# Model class

# This code is heavily based on the Keras Team's example on Convolutional VAEs with only minor modifications.
# The original code can be found at: https://github.com/keras-team/keras-io/blob/master/examples/generative/vae.py

# Author: [fchollet](https://twitter.com/fchollet)
# Licensed under the Apache License 2.0. See LICENSE file in the root directory for more information.


class Sampling(tfkl.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class CVAE(tfk.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tfk.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tfk.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tfk.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            input_data, target_data = data
            z_mean, z_log_var, z = self.encoder(input_data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.losses.binary_crossentropy(target_data, reconstruction), 
                    axis=(1,2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, input_data, training=False):
        _, _, z = self.encoder(input_data)
        reconstructed_image = self.decoder(z)

        return reconstructed_image


In [5]:
# Encoder

models = []

for i in range(MODEL_START, MODEL_START + MODEL_COUNT):
    print(f'Model {i+1+28}')

    image_input = tfkl.Input(shape=consts.INPUT_SHAPE, name='image_input')
    
    x = image_input
    for filters, kernel_size, strides in ENCODER_CONV_CONFIGS[i]:
        x = tfkl.Conv2D(filters=filters, kernel_size=(kernel_size, kernel_size), strides=(strides, strides), activation='relu', padding='same')(x)

    flattened_image = tfkl.Flatten()(x)

    x = flattened_image
    for layer_size in ENCODER_HIDDEN_SIZES[i]:
        x = tfkl.Dense(layer_size, 
                    activation='relu',
                    kernel_initializer='glorot_uniform')(x)
        
    z_mean = tfkl.Dense(LATENT_DIM, name='z_mean', kernel_initializer='glorot_uniform')(x)
    z_log_var = tfkl.Dense(LATENT_DIM, name='z_log_var', kernel_initializer='glorot_uniform')(x)

    z = Sampling()([z_mean, z_log_var])

    encoder = tfk.Model(inputs=image_input, outputs=[z_mean, z_log_var, z], name='encoder')
    encoder.summary()

    # Decoder

    latent_inputs = tfkl.Input(shape=(LATENT_DIM,))

    y = latent_inputs
    for layer_size in DECODER_HIDDEN_SIZES[i]:
        y = tfkl.Dense(layer_size, 
                       activation='relu',
                       kernel_initializer='glorot_uniform')(y)
        
    reconstructed_flattened_image = tfkl.Dense(FLATTENED_SIZES[i], kernel_initializer='glorot_uniform', activation='relu')(y)
    
    y = tfkl.Reshape(FLATTENED_SHAPES[i])(reconstructed_flattened_image)
   
    j = 0
    for filters, kernel_size, strides in DECODER_CONV_CONFIGS[i]:
        if j == len(DECODER_CONV_CONFIGS[i]) - 1:
            y = tfkl.Conv2DTranspose(filters=3, kernel_size=(kernel_size, kernel_size), strides=(strides, strides), activation='sigmoid', padding='same')(y)
        else:
            y = tfkl.Conv2DTranspose(filters=DECODER_CONV_CONFIGS[i][j + 1][0], kernel_size=(kernel_size, kernel_size), strides=(strides, strides), activation='relu', padding='same')(y)
        j += 1

    decoder = tfk.Model(inputs=latent_inputs, outputs=y, name='decoder')
    decoder.summary()

    # Model
        
    model = CVAE(encoder, decoder)
    model.compile(optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE))

    models.append(model)



Model 38
Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 144)          4032      ['image_input[0][0]']         
                                                                                                  
 conv2d_1 (Conv2D)           (None, 16, 16, 224)          290528    ['conv2d[0][0]']              
                                                                                                  
 conv2d_2 (Conv2D)           (None, 4, 4, 448)            2509248   ['conv2d_1[0][0]']            
                                                                                   

In [6]:
# Run model

epochs = 256

i = MODEL_START
for model in models:

    history = igvae.run_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, name=MODEL_NAMES[i], plot=False)
    
    pd.DataFrame(history.history).to_csv(f'res/histories/{MODEL_NAMES[i]}.csv')

    i += 1


Epoch 1/256


2024-04-29 17:00:52.108867: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
2024-04-29 17:00:53.862534: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.94GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
I0000 00:00:1714410054.693379   28172 service.cc:145] XLA service 0x7f7229f61a30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714410054.693440   28172 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-04-29 17:00:54.704373: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714410054.792541   28172 device_compiler.h:188] Compiled cluste

64/64 [==============================] - 32s 180ms/step - loss: 2802.4062 - reconstruction_loss: 2751.6582 - kl_loss: 2.8223 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 2/256
64/64 [==============================] - 6s 94ms/step - loss: 2691.9924 - reconstruction_loss: 2686.5298 - kl_loss: 3.1726 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 3/256
64/64 [==============================] - 6s 95ms/step - loss: 2687.6699 - reconstruction_loss: 2684.5303 - kl_loss: 2.9102 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 4/256
64/64 [==============================] - 6s 94ms/step - loss: 2686.4787 - reconstruction_loss: 2683.9529 - kl_loss: 2.6505 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 5/256
64/64 [==============================] - 6s 95ms/step - loss: 2684.9527 - reconstructi